In [149]:
from langgraph.graph import StateGraph, START,END
from typing import TypedDict
from langgraph.checkpoint.memory import InMemorySaver

In [150]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv("GEMINI_API_KEY")
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite",
                               api_key=api_key)




In [151]:
# define the state

class jokestate(TypedDict):
    topic : str
    joke: str
    explanation: str





In [152]:
def gen_joke(state: jokestate) -> jokestate:
    response = model.invoke(f"Tell me a joke about {state['topic']}")
    return {"joke": response.content}

def exp_joke(state: jokestate) -> jokestate:
    response = model.invoke(f"Explain this joke: {state['joke']}")
    return {"explanation": response.content}

In [153]:
graph = StateGraph(jokestate)


graph.add_node( 'gen_joke',gen_joke )
graph.add_node( 'exp_joke',exp_joke )


graph.add_edge(START, 'gen_joke')
graph.add_edge('gen_joke', 'exp_joke')
graph.add_edge('exp_joke', END)

In [154]:
chekpoint  = InMemorySaver()

workflow=graph.compile(checkpointer=chekpoint) 

In [155]:
congif1= {"configurable":{"thread_id" : "1"}}
inituial_state ={"topic":"CAT"}

workflow.invoke(inituial_state, config=congif1)

{'topic': 'CAT',
 'joke': 'Why did the cat cross the playground?\n\nTo get to the purr-k!',
 'explanation': 'This is a pun! Here\'s the breakdown:\n\n*   **The Setup:** "Why did the cat cross the playground?" sets up a simple question, similar to the classic "Why did the chicken cross the road?" joke.\n*   **The Punchline:** "To get to the purr-k!" is the wordplay.\n    *   "Purr-k" sounds like "park."\n    *   Cats purr, so the joke uses that word association to create humor.\n\n**In essence, the joke plays on the similarity in sound between "purr" (what a cat does) and "park" (a place).** It\'s a silly, lighthearted pun.'}

In [156]:
congif2= {"configurable":{"thread_id" : "2"}}
inituial_state ={"topic":"DOG"}

workflow.invoke(inituial_state, config=congif2)

{'topic': 'DOG',
 'joke': 'Why did the dog cross the playground? \n\nTo get to the *bark*ing lot!',
 'explanation': 'The joke plays on a pun, which is a play on words. Here\'s the breakdown:\n\n*   **"Why did the dog cross the playground?"** - This sets up the question, creating anticipation for the punchline.\n*   **"To get to the *bark*ing lot!"** -\n    *   The word "bark" has two meanings:\n        *   It\'s the sound a dog makes.\n        *   It sounds like "park," which is a place where people can leave their cars.\n    *   The joke is funny because it leads the listener to believe the dog is going to a park, but then the pun reveals the dog is going to the "barking lot," a play on the word "parking" and the sound a dog makes.'}

In [157]:
workflow.get_state(congif1)

StateSnapshot(values={'topic': 'CAT', 'joke': 'Why did the cat cross the playground?\n\nTo get to the purr-k!', 'explanation': 'This is a pun! Here\'s the breakdown:\n\n*   **The Setup:** "Why did the cat cross the playground?" sets up a simple question, similar to the classic "Why did the chicken cross the road?" joke.\n*   **The Punchline:** "To get to the purr-k!" is the wordplay.\n    *   "Purr-k" sounds like "park."\n    *   Cats purr, so the joke uses that word association to create humor.\n\n**In essence, the joke plays on the similarity in sound between "purr" (what a cat does) and "park" (a place).** It\'s a silly, lighthearted pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-4e1f-6189-8002-3905d3672c7f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T18:42:19.976333+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-3e85-67fa-8001-

In [158]:
list(workflow.get_state_history(congif1))

[StateSnapshot(values={'topic': 'CAT', 'joke': 'Why did the cat cross the playground?\n\nTo get to the purr-k!', 'explanation': 'This is a pun! Here\'s the breakdown:\n\n*   **The Setup:** "Why did the cat cross the playground?" sets up a simple question, similar to the classic "Why did the chicken cross the road?" joke.\n*   **The Punchline:** "To get to the purr-k!" is the wordplay.\n    *   "Purr-k" sounds like "park."\n    *   Cats purr, so the joke uses that word association to create humor.\n\n**In essence, the joke plays on the similarity in sound between "purr" (what a cat does) and "park" (a place).** It\'s a silly, lighthearted pun.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-4e1f-6189-8002-3905d3672c7f'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T18:42:19.976333+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-3e85-67fa-8001

In [159]:
workflow.get_state(congif2)

StateSnapshot(values={'topic': 'DOG', 'joke': 'Why did the dog cross the playground? \n\nTo get to the *bark*ing lot!', 'explanation': 'The joke plays on a pun, which is a play on words. Here\'s the breakdown:\n\n*   **"Why did the dog cross the playground?"** - This sets up the question, creating anticipation for the punchline.\n*   **"To get to the *bark*ing lot!"** -\n    *   The word "bark" has two meanings:\n        *   It\'s the sound a dog makes.\n        *   It sounds like "park," which is a place where people can leave their cars.\n    *   The joke is funny because it leads the listener to believe the dog is going to a park, but then the pun reveals the dog is going to the "barking lot," a play on the word "parking" and the sound a dog makes.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-6a7e-66b6-8002-cc1b92fd4030'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T18:42:22.951391+00:00', p

In [160]:
list(workflow.get_state_history(congif2))

[StateSnapshot(values={'topic': 'DOG', 'joke': 'Why did the dog cross the playground? \n\nTo get to the *bark*ing lot!', 'explanation': 'The joke plays on a pun, which is a play on words. Here\'s the breakdown:\n\n*   **"Why did the dog cross the playground?"** - This sets up the question, creating anticipation for the punchline.\n*   **"To get to the *bark*ing lot!"** -\n    *   The word "bark" has two meanings:\n        *   It\'s the sound a dog makes.\n        *   It sounds like "park," which is a place where people can leave their cars.\n    *   The joke is funny because it leads the listener to believe the dog is going to a park, but then the pun reveals the dog is going to the "barking lot," a play on the word "parking" and the sound a dog makes.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f08507e-6a7e-66b6-8002-cc1b92fd4030'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-08-29T18:42:22.951391+00:00', 